In [3]:
from pathlib import Path
import os

try:
    import wandb
except:
    if os.environ['COLAB_RELEASE_TAG']:
        print("Found Colab Environment")
        from google.colab import drive
        drive.mount('/content/drive')
        from google.colab import auth
        auth.authenticate_user()

        %pip install -q tensorflow==2.10.0 wandb python-dotenv tensorboard_plugin_profile tensorflow_io==0.27.0
        exit()
    elif Path().cwd().name == 'Mushroom-Classifier':
        print("Found Other Environment")
        %pip install -q tensorflow==2.10.0 wandb python-dotenv tensorboard_plugin_profile tensorflow_io==0.27.0
        exit()
    else:
        print('Please run this notebook from the root of the repository')
        exit()

%cd /content/drive/MyDrive/Mushroom-Classifier

[Errno 2] No such file or directory: '/content/drive/MyDrive/Mushroom-Classifier'
/home/broug/Desktop/Mushroom-Classifier/training


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math, re, os, pickle
import tensorflow as tf
from datetime import datetime
import wandb
from wandb.keras import WandbCallback, WandbModelCheckpoint
import numpy as np
from matplotlib import pyplot as plt
# from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from src.models.swintransformer import SwinTransformer
# from src.optimizers import lion
# from prefect import task, flow

print(f"Tensorflow version {tf.__version__}")
AUTO = tf.data.experimental.AUTOTUNE
np.set_printoptions(threshold=15, linewidth=80)

from config import GCFG, CFG

CFG2 = GCFG()

Tensorflow version 2.14.0


In [3]:
save_time = datetime.now().strftime('%m%d-%H%M')
log_dir = f"{CFG2.GCS_REPO}/logs/{CFG2.MODEL}/{save_time}"

# wandb.tensorboard.patch(root_logdir=log_dir + "/tf")
wandb.init(project="Mushroom-Classifier", tags=[CFG2.MODEL, CFG2.OPT, CFG2.LR_SCHED, str(CFG2.IMAGE_SIZE[0])])

wandb: Currently logged in as: g-broughton. Use `wandb login --relogin` to force relogin


In [4]:
!export TPU_NAME=local

In [3]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
strategy = tf.distribute.TPUStrategy(cluster_resolver)

CFG2.REPLICAS = strategy.num_replicas_in_sync
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2023-10-26 13:26:08.078897: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5623a163ea00 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
2023-10-26 13:26:08.078939: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): TPU, 2a886c8
2023-10-26 13:26:08.078950: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): TPU, 2a886c8
2023-10-26 13:26:08.078960: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): TPU, 2a886c8
2023-10-26 13:26:08.078969: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): TPU, 2a886c8
2023-10-26 13:26:08.078979: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (4): TPU, 2a886c8
2023-10-26 13:26:08.078988: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (5): TPU, 2a886c8
2023-10-26 13:26:08.078998: I tensorflow/compiler/xla/service/service.cc:176]   StreamEx

INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [3]:
%run train.py

2023-10-26 13:32:57.303378: I tensorflow/compiler/xla/stream_executor/tpu/tpu_initializer_helper.cc:242] Libtpu path is: libtpu.so
D1026 13:32:57.467209113   45943 config.cc:175]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D1026 13:32:57.467235345   45943 config.cc:175]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D1026 13:32:57.467240557   45943 config.cc:175]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D1026 13:32:57.467244820   45943 config.cc:175]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D1026 13:32:57.467249466   45943 config.cc:175]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D1026 13:32:57.467253614   45943 config.cc:175]                        gRPC EXPERIMENT event_engine_client                 OFF (default:OFF)
D1026 13:32:57.46725781

Tensorflow version 2.14.0


wandb: Currently logged in as: g-broughton. Use `wandb login --relogin` to force relogin


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2023-10-26 13:33:09.995469: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563d7e8ddcc0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
2023-10-26 13:33:09.995506: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): TPU, 2a886c8
2023-10-26 13:33:09.995518: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): TPU, 2a886c8
2023-10-26 13:33:09.995527: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): TPU, 2a886c8
2023-10-26 13:33:09.995537: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): TPU, 2a886c8
2023-10-26 13:33:09.995546: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (4): TPU, 2a886c8
2023-10-26 13:33:09.995556: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (5): TPU, 2a886c8
2023-10-26 13:33:09.995570: I tensorflow/compiler/xla/service/service.cc:176]   StreamEx

INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

2023-10-26 13:33:14.735 | INFO     | __main__:main:25 - Number of accelerators: 
2023-10-26 13:33:14.840 | DEBUG    | __main__:main:50 - Training data shapes:
2023-10-26 13:33:16.785 | DEBUG    | __main__:main:52 - ((64, 3, 224, 224), (64,))
2023-10-26 13:33:17.031 | DEBUG    | __main__:main:52 - ((64, 3, 224, 224), (64,))
2023-10-26 13:33:17.168 | DEBUG    | __main__:main:52 - ((64, 3, 224, 224), (64,))
2023-10-26 13:33:17.253502: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-10-26 13:33:18.390 | DEBUG    | __main__:main:53 - Training data label examples: [154  41 146 ...  59 421 421]
2023-10-26 13:33:18.392 | DEBUG    | __main__:main:5

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/broug/Mushroom-Classifier/training/base_models/swin_large_224/base_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [27]:
# # Detect hardware
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# except ValueError:  # If TPU not found
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy()
# CFG2.REPLICAS = strategy.num_replicas_in_sync
# print("Number of accelerators: ", strategy.num_replicas_in_sync)

## Visualization Utils

In [6]:
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return class_dict[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(class_dict[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                class_dict[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)

def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]

    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows

    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))

    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else class_dict[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)

    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(class_dict)))
    ax.set_xticklabels(class_dict, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(class_dict)))
    ax.set_yticklabels(class_dict, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if titlestring != "":
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()

def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title(f'model {title}')
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    wandb.log({"chart": plt})
    path = CFG.ROOT / "images" / CFG.MODEL
    path.mkdir(exist_ok=True)
    plt.savefig(path / f'{title}-{save_time}.png')

In [7]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)  # image format uint8 [0,255]
    image = tf.reshape(image, [*CFG.IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image


def read_labeled_tfrecord(example):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'dataset': tf.io.FixedLenFeature([], tf.int64),
        'longitude': tf.io.FixedLenFeature([], tf.float32),
        'latitude': tf.io.FixedLenFeature([], tf.float32),
        'norm_date': tf.io.FixedLenFeature([], tf.float32),
        'class_priors': tf.io.FixedLenFeature([], tf.float32),
        'class_id': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = decode_image(example['image'])
    label = tf.cast(example['class_id'], tf.int32)
    return image, label


def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.cache()
    dataset = dataset.shuffle(CFG.BATCH_SIZE * 10)
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord, num_parallel_calls=AUTO) # if labeled else read_unlabeled_tfrecord
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    # image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
     # the training dataset must repeat for several epochs
    dataset = dataset.batch(CFG.BATCH_SIZE)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(CFG.BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [8]:
GCS_PATH_SELECT = {
    192: f'{CFG2.GCS_REPO}/tfrecords-jpeg-192x192',
    224: f'{CFG2.GCS_REPO}/tfrecords-jpeg-224x224v2',
    384: f'{CFG2.GCS_REPO}/tfrecords-jpeg-384x384',
    512: f'{CFG2.GCS_REPO}/tfrecords-jpeg-512x512',
}
GCS_PATH = GCS_PATH_SELECT[CFG2.IMAGE_SIZE[0]]
TRAINING_FILENAMES = tf.io.gfile.glob(f'{GCS_PATH}/train*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(f'{GCS_PATH}/val*.tfrec')

class_dict = pickle.load(open('src/class_dict.pkl', 'rb'))

CFG2.NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
CFG2.NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)

CFG = CFG(REPLICAS=CFG2.REPLICAS, NUM_TRAINING_IMAGES=CFG2.NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES=CFG2.NUM_VALIDATION_IMAGES)

In [10]:
# data dump
print("Training data shapes:")
for image, label in get_training_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())
print("Validation data shapes:")
for image, label in get_validation_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Validation data label examples:", label.numpy())

Training data shapes:
(64, 224, 224, 3) (64,)
(64, 224, 224, 3) (64,)
(64, 224, 224, 3) (64,)


2023-10-26 12:27:22.276860: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Training data label examples: [222 445 194 ... 279 311 271]
Validation data shapes:
(64, 224, 224, 3) (64,)
(64, 224, 224, 3) (64,)
(64, 224, 224, 3) (64,)


2023-10-26 12:27:24.545559: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Validation data label examples: [ 97 221 410 ...  84 177 246]


In [11]:
# Peek at training data
training_dataset = get_training_dataset()
training_dataset = training_dataset.unbatch().batch(20)
train_batch = iter(training_dataset)

In [ ]:
# run this cell again for next set of images
# display_batch_of_images(next(train_batch))

you can select from these models:
- swin_tiny_224
- swin_small_224
- swin_base_224
- swin_base_384
- swin_large_224
- swin_large_384

In [11]:
def make_callbacks(CFG):
    # options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_sparse_categorical_crossentropy",
            patience=CFG.ES_PATIENCE,
            verbose=1,
            restore_best_weights=True,
        ),
        # tf.keras.callbacks.TensorBoard(log_dir=log_dir + "/tf", profile_batch=(50, 250)),
        tf.keras.callbacks.CSVLogger(
            filename=f'{CFG.GCS_REPO}/logs/{save_time}-csv_log.csv',
            separator=",",
            append=False,
        ),
        # wandb.keras.WandbMetricsLogger(log_freq='batch'),
        # wandb.keras.WandbModelCheckpoint(
        #     str(CFG.ROOT / 'models' / CFG.MODEL / f"{save_time}.h5"),
        #     monitor='val_loss', verbose=1, save_best_only=True,
        #     save_weights_only=True, options=options,
        # )
    ]
    return callbacks

In [38]:
%load_ext tensorboard

In [12]:
with strategy.scope():
    img_adjust_layer = tf.keras.layers.Lambda(
        lambda data: tf.keras.applications.imagenet_utils.preprocess_input(
            tf.cast(data, tf.float32), mode="torch"),
        input_shape=[*CFG.IMAGE_SIZE, 3])
    pretrained_model = tf.keras.models.load_model(
        CFG.ROOT / 'base_models' / CFG.MODEL / 'base_model', compile=False
    )
    model = tf.keras.Sequential([
        img_adjust_layer,
        pretrained_model,
        tf.keras.layers.Dense(len(class_dict), activation='softmax')
    ])

    top3_acc = tf.keras.metrics.SparseTopKCategoricalAccuracy(
        k=3, name='sparse_top_3_categorical_accuracy'
    )
    lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=CFG.LR_START,
        decay_steps=CFG.DECAY_STEPS
    )
    optimizer = tf.keras.optimizers.Adam(lr_decayed_fn)

2023-10-26 12:27:41.132542: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-26 12:27:41.177060: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-10-26 12:27:41.448547: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:27:41.599988: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


In [13]:
model.compile(
    optimizer=optimizer,  # lion.Lion(learning_rate=lr_decayed_fn),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy', top3_acc],
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 swin_large_224 (SwinTransf  (None, 1536)              195331616 
 ormerModel)                                                     
                                                                 
 dense (Dense)               (None, 467)               717779    
                                                                 
Total params: 196049395 (749.15 MB)
Trainable params: 195713255 (746.59 MB)
Non-trainable params: 336140 (2.56 MB)
_________________________________________________________________


In [12]:
# service_addr = tpu.get_master().replace(':8470', ':8466')
# print(service_addr)
# %tensorboard --logdir={log_dir + "/tf"}

In [14]:
wandb.config = CFG

In [15]:
history = model.fit(
    get_training_dataset(),
    steps_per_epoch=CFG.STEPS_PER_EPOCH,
    epochs=CFG.EPOCHS,
    validation_data=get_validation_dataset(),
    validation_steps=CFG.VALIDATION_STEPS,
    callbacks=make_callbacks(CFG)
)

Epoch 1/20


2023-10-26 12:29:32.891956: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:29:32.916138: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:29:32.939980: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:29:32.963671: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:29:32.987170: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:29:33.010724: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:29:33.036115: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

  1/189 [..............................] - ETA: 11:33:48 - loss: 6.4501 - sparse_categorical_accuracy: 0.0000e+00 - sparse_top_3_categorical_accuracy: 0.0000e+00

2023-10-26 12:33:14.549158: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:33:14.549582: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:33:14.552268: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:33:14.552781: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 12:33:14.554408: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


189/189 [==============================] - ETA: 0s - loss: 3.4266 - sparse_categorical_accuracy: 0.2906 - sparse_top_3_categorical_accuracy: 0.4623

...

189/189 [==============================] - 62s 327ms/step - loss: 0.3755 - sparse_categorical_accuracy: 0.8929 - sparse_top_3_categorical_accuracy: 0.9773 - val_loss: 0.8220 - val_sparse_categorical_accuracy: 0.7617 - val_sparse_top_3_categorical_accuracy: 0.9160
Epoch 19/20
189/189 [==============================] - ETA: 0s - loss: 0.2846 - sparse_categorical_accuracy: 0.9225 - sparse_top_3_categorical_accuracy: 0.9878WARNING:tensorflow:Early stopping conditioned on metric `val_sparse_categorical_crossentropy` which is not available. Available metrics are: loss,sparse_categorical_accuracy,sparse_top_3_categorical_accuracy,val_loss,val_sparse_categorical_accuracy,val_sparse_top_3_categorical_accuracy


2023-10-26 12:53:43.460830: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


189/189 [==============================] - 62s 329ms/step - loss: 0.2846 - sparse_categorical_accuracy: 0.9225 - sparse_top_3_categorical_accuracy: 0.9878 - val_loss: 0.9960 - val_sparse_categorical_accuracy: 0.7402 - val_sparse_top_3_categorical_accuracy: 0.8730
Epoch 20/20
189/189 [==============================] - ETA: 0s - loss: 0.3209 - sparse_categorical_accuracy: 0.9206 - sparse_top_3_categorical_accuracy: 0.9830WARNING:tensorflow:Early stopping conditioned on metric `val_sparse_categorical_crossentropy` which is not available. Available metrics are: loss,sparse_categorical_accuracy,sparse_top_3_categorical_accuracy,val_loss,val_sparse_categorical_accuracy,val_sparse_top_3_categorical_accuracy


2023-10-26 12:54:44.654317: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


189/189 [==============================] - 61s 323ms/step - loss: 0.3209 - sparse_categorical_accuracy: 0.9206 - sparse_top_3_categorical_accuracy: 0.9830 - val_loss: 0.9886 - val_sparse_categorical_accuracy: 0.7461 - val_sparse_top_3_categorical_accuracy: 0.8867


In [17]:
model.save('model')

[autoreload of jupyter_server.serverapp failed: Traceback (most recent call last):
  File "/home/broug/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/broug/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/broug/.local/lib/python3.10/site-packages/jupyter_server/serverapp.py", line 77, in <module>
    from jupyter_server.auth.identity import (
  File "/home/broug/.local/lib/python3.10/site-packages/jupyter_server/auth/identity.py", line 28, in <module>
    from .utils import get_anonymo

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [ ]:
art = wandb.Artifact(
    'model',
    type='model')
art.add_file(str(CFG.ROOT / 'models' / CFG.MODEL / f"{time}.h5"))
wandb.log_artifact(art)

In [15]:
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 211)
display_training_curves(history.history['sparse_categorical_accuracy'], history.history['val_sparse_categorical_accuracy'], 'accuracy', 212)

<ipython-input-6-0b107be7bb99>:92: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax = plt.subplot(subplot)
/usr/local/lib/python3.10/dist-packages/plotly/matplotlylib/renderer.py:647: UserWarning:

Looks like the annotation(s) you are trying 
to draw lies/lay outside the given figure size.

Therefore, the resulting Plotly figure may not be 
large enough to view the full text. To adjust 
the size of the figure, use the 'width' and 
'height' keys in the Layout object. Alternatively,
use the Margin object to adjust the figure's margins.

/usr/local/lib/python3.10/dist-packages/plotly/matplotlylib/renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



<Figure size 640x480 with 0 Axes>

In [16]:
wandb.finish()